A variable is any name that holds a value or object, and it can be either
mutable and immutable—that is, it can either "mutate" (change) or it can't.

## Undeclared variables

Within a `def` function or a REPL environment, Mojo allows you to create a
variable with just a name and a value. For example:

In [1]:
name = "Sam"

This is called an undeclared variable because it doesn't use the `var` or
`let` declarations, described below. Undeclared variables are always mutable
(you can change the value later), which provides the same behavior as Python.

However, undeclared variables not allowed in an `fn` function.

## Declared mutable `var` and immutable `let` variables

You can declare a variable as mutable with `var` or immutable
with `let`. For example:

In [2]:
var name = "Sam"  # This can be assigned a new value later
let user_id = 42  # This can be assigned a value only once

The `name` variable is mutable, so you can change it later, but if you try to
change `id` after it is defined, the Mojo compiler won't allow it.

**Note:** All variables in an `fn` function must be declared with `var` or
`let`. These declarations are optional in a `def` function.

Using `var` helps prevent runtime errors caused by typos. For example, if you
misspell the name of an [undeclared variable](#undeclared-variables), Mojo
simply instantiates a new variable using the misspelled name. But when all
mutable variables must be first declared with `var` (which is the case inside
an `fn` function), then misspellings such as the following are caught by the
compiler:

```
var name = "Sam"
# Somewhere later...
nane = "Sammy"  # This is not allowed in an `fn` function
```

So, although using `var` is optional in a `def` function, the benefit is
realized only when used inside an `fn` function, where the Mojo compiler will
flag undeclared variables (such as the above `nane`) as unknown declarations.

Whereas, declaring an immutable `let` variable (also known as a "constant") is
useful in any situation where you want to avoid bugs that can occur when a
value is supposed to remain the same but is accidentally changed anyway. By
using `let`, the compiler catches these mistakes (in `def` and `fn` functions),
so they don't cause errors at runtime. Also, because the `let` value is
guaranteed to not change at runtime, the compiler can make some performance
optimizations.

**Note:** When using Mojo in a REPL environment, top-level variables (variables
that live outside a function or struct) are treated like variables in a `def`
(they do not require `var` or `let` declarations). This matches the Python REPL
behavior to allow simple script-style programming.

## Type specifiers

Although Mojo allows for dynamic types (like Python), it also supports static
type declarations for variables. However, you **must** specify the types for
arguments and return values in an [`fn`
function](/mojo/manual/basics/functions.html).

To specify the type for a variable, add a colon and type name after the
name. For example:

In [3]:
var name: String = "Sam"

:::{.callout-note}

**Note:** You must declare `var` or `let` in order to specify the type
(undeclared variables do not support static type specifiers).

:::

When a type has a constructor that takes a single value, like a `String` or
`Int`, you can initialize it in either of these ways:

In [3]:
var name1: String = "Sam"
var name2 = String("Sam")

Both lines invoke the constructor and both variables are statically typed.

Type specifiers also allow late initialization (notice that the `z` variable is
first declared with its type, but the value is assigned later):

In [4]:
fn your_function(x: Int):
    let z: Float32
    if x != 0:
        z = 1.0
    else:
        z = foo()
    print(z)

fn foo() -> Float32:
    return 3.14

your_function(42)

1.0


## Variable scopes

Variables declared with `var` and `let` are bound by **lexical scoping**. This
means that nested code blocks can read and modify variables defined in an
outer scope. Conversely, an outer scope cannot read variables defined in an
inner scope at all.

For example, the `if` code block shown here creates an inner scope where outter
variables are accessible to read/write, but any new variables do not live
beyond the scope of the `if` block:

In [5]:
def lexical_scopes():
    let num = 10
    var dig = 1
    if True:
        print("num:", num)  # Reads the outer-scope "num"
        let num = 20        # Creates new inner-scope "num"
        print("num:", num)  # Reads the inner-scope "num"
        dig = 2             # Edits the outer-scope "dig"
    print("num:", num)      # Reads the outer-scope "num"
    print("dig:", dig)      # Reads the outer-scope "dig"

lexical_scopes()

num: 10
num: 20
num: 10
dig: 2


The lifetime of the inner `num` ends exactly where the `if` code block ends,
because that's the scope in which the variable was defined.

This is in contrast to undeclared variables (those without the `var` or `let`
keyword), which use **function-level scoping** (consistent with Python variable
behavior). That means, when you change the value of an undeclared variable
inside the `if` block, it actually changes the value for the entire function.

For example, here's the same code but *without* the `let` declarations:

In [6]:
def function_scopes():
    num = 1
    if num == 1:
        print(num)   # Reads the function-scope "num"
        num = 2      # Updates the function-scope variable
        print(num)   # Reads the function-scope "num"
    print(num)       # Reads the function-scope "num"

function_scopes()

1
2
2


Now, the last `print()` function sees the updated `num` value from the inner
scope, because undeclared variables (Python-style variables) use function-level
scope (instead of lexical scope).